In [1]:
import pandas as pd
import numpy as np
import glob 
import os 
import PIL
import scipy.linalg as la
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import image
import warnings
warnings.filterwarnings("ignore")

In [2]:
def im2matrix(folder):
    data=[]
    for i in folder:
        img_mat=image.imread(i)
        img_mat=img_mat.flatten()
        data.append(img_mat)
    data=np.mat(data)
    return(data)

neutral_face_train='upload_dataset_100/*a.jpg'
neutral_face_train=glob.glob(neutral_face_train)
neutral_data_train=im2matrix(neutral_face_train)
neutral_data_train=neutral_data_train/255
neutral_data_train=neutral_data_train.T

smiling_face_train='upload_dataset_100/*b.jpg'
smiling_face_train=glob.glob(smiling_face_train)
smiling_data_train=im2matrix(smiling_face_train)
smiling_data_train=smiling_data_train/255
smiling_data_train=smiling_data_train.T

neutral_face_test='upload_dataset_71/*a.jpg'
neutral_face_test=glob.glob(neutral_face_test)
neutral_data_test=im2matrix(neutral_face_test)
neutral_data_test=neutral_data_test/255
neutral_data_test=neutral_data_test.T

smiling_face_test='upload_dataset_71/*b.jpg'
smiling_face_test=glob.glob(smiling_face_test)
smiling_data_test=im2matrix(smiling_face_test)
smiling_data_test=smiling_data_test/255
smiling_data_test=smiling_data_test.T

# 1.

### Neutral

In [3]:
N=neutral_data_train.shape[1]
mean_vec=np.mean(neutral_data_train,axis=1)
A=neutral_data_train.copy()
for i in range(N): 
    A[:,i]=A[:,i]-mean_vec
svd=A.T*A
evals,evecs=la.eig(svd)
evecs_new=np.zeros(A.shape)
for i in range(N):
    Aeigen=A*evecs[:,i].reshape((-1,1))
    u=(Aeigen/np.linalg.norm(Aeigen))
    evecs_new[:,i]=u.ravel()

In [ ]:
for i in range(10):
    plt.imshow(evecs_new[:,i].reshape(193, 162),cmap ='gray')
    plt.title('Representative Eigenfaces(N)-{}'.format(i+1))
    plt.savefig('Representative Eigenfaces(N)-{}'.format(i+1),bbox_inches='tight')

In [ ]:
singular_values=np.sqrt(evals[evals>0])
sort_index=np.argsort(singular_values)[::-1]
singular_values=singular_values[sort_index]

plt.xlabel('Singular Value')
plt.ylabel('Number')
sns.distplot(singular_values,kde=False);

### Smiling

In [7]:
N=smiling_data_train.shape[1]
mean_vec_s=np.mean(smiling_data_train,axis=1)
B=smiling_data_train.copy()
for i in range(N): 
    B[:,i]=B[:,i]-mean_vec_s
svd=B.T*B
evals,evecs=la.eig(svd)
evecs_new_s=np.zeros(B.shape)
for i in range(N):
    Beigen=B*evecs[:,i].reshape((-1,1))
    u=(Beigen/np.linalg.norm(Beigen))
    evecs_new_s[:,i]=u.ravel()

In [ ]:
for i in range(10):
    plt.imshow(evecs_new_s[:,i].reshape(193, 162),cmap ='gray')
    plt.title('Representative Eigenfaces(S)-{}'.format(i+1))
    plt.savefig('Representative Eigenfaces(S)-{}'.format(i+1),bbox_inches='tight')

In [ ]:
singular_values=np.sqrt(evals[evals>0])
sort_inex=np.argsort(singular_values)[::-1]
singular_values=singular_values[sort_index]

plt.xlabel('Singular Value')
plt.ylabel('Number')
sns.distplot(singular_values,kde=False);

# 2.

In [15]:
def mse(a,b):
    dif=a-b
    dif=dif.reshape(-1).tolist()[0]
    dif=[i**2 for i in dif]
    return(np.mean(dif))

def reconstruct(original,mean_vec,evecs_new,num_pc=None):
    if num_pc: 
        use_evecs=evecs_new[:,0:num_pc]
    else:
        num_pc=evecs_new.shape[1]
        use_evecs=evecs_new
    w=np.zeros(num_pc)
    for i in range(num_pc):
        w[i]=np.dot(use_evecs[:,i],original)
    re=use_evecs*w.reshape((1,-1))
    re=np.sum(re,axis=1)
    re=255*(mean_vec+re.reshape((-1, 1)))
    return(re)

In [ ]:
neutral_original=A[:,0]
orig_pic_vec=(neutral_original+mean_vec)*255
test_pcs=[10,20,30,40,50,60,70,80,90,100]

mse_data=[]
for i,num_pc in enumerate(test_pcs): 
    reconstruction=reconstruct(neutral_original,mean_vec,evecs_new,num_pc=num_pc)
    mse_val=mse(orig_pic_vec, reconstruction)
    mse_data.append([num_pc,mse_val])
    plt.imshow(reconstruction.reshape(193,162),cmap ='gray')
    plt.title('P={}'.format(num_pc))
    plt.savefig('P={},N'.format(num_pc),bbox_inches='tight')

In [ ]:
mse_data=pd.DataFrame(mse_data,columns=["Number of PC",'MSE'])
sns.scatterplot(data=mse_data,x="Number of PC",y='MSE')

# 3.

In [ ]:
smiling_original=B[:,0]
orig_pic_vec=(smiling_original+mean_vec_s)*255
test_pcs=[10,20,30,40,50,60,70,80,90,100]

mse_data=[]
for i,num_pc in enumerate(test_pcs): 
    reconstruction=reconstruct(smiling_original,mean_vec_s,evecs_new_s,num_pc=num_pc)
    mse_val=mse(orig_pic_vec, reconstruction)
    mse_data.append([num_pc,mse_val])
    plt.imshow(reconstruction.reshape(193,162),cmap ='gray')
    plt.title('P={}'.format(num_pc))
    plt.savefig('P={},S'.format(num_pc),bbox_inches='tight')

In [ ]:
mse_data=pd.DataFrame(mse_data,columns=["Number of PC",'MSE'])
sns.scatterplot(data=mse_data,x="Number of PC",y='MSE')

# 4.

### Neutral

In [ ]:
C=neutral_data_test.copy()
mean_vec_c=np.mean(C,axis=1)

neutral_original=C[:,0]
orig_pic_vec=neutral_original*255
test_pcs=[10,20,30,40,50,60,70,80,90,100]

mse_data=[]
for i,num_pc in enumerate(test_pcs): 
    reconstruction=reconstruct(neutral_original,mean_vec_c,evecs_new,num_pc=num_pc)
    mse_val=mse(orig_pic_vec, reconstruction)
    mse_data.append([num_pc,mse_val])
    plt.imshow(reconstruction.reshape(193,162),cmap ='gray')
    plt.title('P={}'.format(num_pc))
    plt.savefig('P={},Nt'.format(num_pc),bbox_inches='tight')

In [ ]:
mse_data=pd.DataFrame(mse_data,columns=["Number of PC",'MSE'])
sns.scatterplot(data=mse_data,x="Number of PC",y='MSE')

### Smiling

In [ ]:
D=smiling_data_test.copy()
mean_vec_d=np.mean(D,axis=1)
smiling_original=D[:,0]
orig_pic_vec=smiling_original*255
test_pcs=[10,20,30,40,50,60,70,80,90,100]

mse_data=[]
for i,num_pc in enumerate(test_pcs): 
    reconstruction=reconstruct(smiling_original,mean_vec_d,evecs_new_s,num_pc=num_pc)
    mse_val=mse(orig_pic_vec, reconstruction)
    mse_data.append([num_pc,mse_val])
    plt.imshow(reconstruction.reshape(193,162),cmap ='gray')
    plt.title('P={}'.format(num_pc))
    plt.savefig('P={},St'.format(num_pc),bbox_inches='tight')

In [ ]:
mse_data=pd.DataFrame(mse_data,columns=["Number of PC",'MSE'])
sns.scatterplot(data=mse_data,x="Number of PC",y='MSE')

# 5.

In [24]:
neutral_face_test_5='upload_dataset_60/*a.jpg'
neutral_face_test_5=glob.glob(neutral_face_test_5)
neutral_data_test_5=im2matrix(neutral_face_test_5)
neutral_data_test_5=neutral_data_test_5/255
neutral_data_test_5=neutral_data_test_5.T

smiling_face_test_5='upload_dataset_60/*b.jpg'
smiling_face_test_5=glob.glob(smiling_face_test_5)
smiling_data_test_5=im2matrix(smiling_face_test_5)
smiling_data_test_5=smiling_data_test_5/255
smiling_data_test_5=smiling_data_test_5.T

In [ ]:
E=neutral_data_test_5.copy()
F=smiling_data_test_5.copy()
mean_vec_e=np.mean(E,axis=1)
mean_vec_f=np.mean(F,axis=1)
flag_e=[]
flag_f=[]
for _ in range(60):
    neutral_original=E[:,_]
    smiling_original=F[:,_]
    neutral_original=neutral_original-mean_vec_e
    smiling_original=smiling_original-mean_vec_f
    orig_pic_vec_n=neutral_original*255
    orig_pic_vec_s=smiling_original*255
    test_pcs_n=11
    test_pcs_s=13
    
    e_reconstruction_n=reconstruct(neutral_original,mean_vec_e,evecs_new,num_pc=60)
    e_reconstruction_s=reconstruct(neutral_original,mean_vec_e,evecs_new_s,num_pc=60)
    mse_val_1=mse(orig_pic_vec_n,e_reconstruction_n)
    mse_val_2=mse(orig_pic_vec_n,e_reconstruction_s)
    if mse_val_1>mse_val_2:
        flag_e.append(0)
    else:
        flag_e.append(1)
    
    f_reconstruction_n=reconstruct(smiling_original,mean_vec_f,evecs_new,num_pc=60)
    f_reconstruction_s=reconstruct(smiling_original,mean_vec_f,evecs_new_s,num_pc=60)
    mse_val_3=mse(orig_pic_vec_s,f_reconstruction_n)
    mse_val_4=mse(orig_pic_vec_s,f_reconstruction_s)
    if mse_val_3>mse_val_4:
        flag_f.append(0)
    else:
        flag_f.append(1)
    
success_rate_n=sum(flag_e)/60
success_rate_s=(60-sum(flag_f))/60
print(success_rate_n)
print(success_rate_s)